<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем библиотеки

In [12]:
import os
import numpy as np
import pandas as pd
import re
import nltk
from tqdm import tqdm
tqdm.pandas()
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, roc_auc_score
RND_ST = 12345
cv_counts = 2
import nltk

In [13]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en
import spacy

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 1.0 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Грузим датасет

In [14]:
pth = '/datasets/toxic_comments.csv'
if os.path.exists(pth):
    df = pd.read_csv(pth,index_col=[0])
else:
    print('Путь не таков')

Выводим общую инфу по датасету

In [15]:
def info_func(df):
    print('Общая информация')
    print('')
    print(df.info())
    print('')
    print(df.describe())
    print('')
    for column in df.columns:
        print('Уникальные значения столбца', column)
        print('')
        print(df[column].unique())
        print('')
    print('Кол-во строк дубликатов')
    print('')
    print(df[df.duplicated()].count().sort_values(ascending=False))
    print('')
    print('Кол-во пропущенных значений')
    print('')
    print(df.isna().sum().sort_values(ascending=False))
    print('')
    print('Процент пропущенных значений')
    print('')
    print((df.isna().sum()/len(df)*100).sort_values(ascending=False))
    print('')
    print(df.head(-5))

In [16]:
info_func(df)

Общая информация

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB
None

               toxic
count  159292.000000
mean        0.101612
std         0.302139
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000

Уникальные значения столбца text

["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"
 "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"
 "Hey man, I'm really not trying to edit war. It's just tha

Смотрим отношение токсичных комментов ко всему объему

In [17]:
display(df['toxic'].value_counts())
ratio = df['toxic'].value_counts()[0] / df['toxic'].value_counts()[1]
print('Процент токсичности')
ratio

0    143106
1     16186
Name: toxic, dtype: int64

Процент токсичности


8.841344371679229

Присутствует явный дисбаланс.

Приведем слова к начальной форме и избавим от лишних символов

In [18]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatize(text):
    text = text.lower()
    doc = nlp(text)
    lemm = " ".join([token.lemma_ for token in doc])
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm) 
    return " ".join(cleared_text.split())

sentence1 = "The striped bats are hanging on their feet for best"
sentence2 = "you should be ashamed of yourself went worked"
df_my = pd.DataFrame([sentence1, sentence2], columns = ['text'])
print(df_my)


print(df_my['text'].apply(lemmatize))

                                                text
0  The striped bats are hanging on their feet for...
1      you should be ashamed of yourself went worked
0    the stripe bat be hang on their foot for good
1        you should be ashamed of yourself go work
Name: text, dtype: object


In [19]:
df['lemm'] = df['text'].progress_apply(lemmatize)

df = df.drop(['text'], axis=1)

100%|██████████| 159292/159292 [16:09<00:00, 164.23it/s]


Создадим признаки и цель. Тренировочную, валидационную и тестовую выборки.

In [20]:
target = df['toxic']
features = df.drop(['toxic'], axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.5, random_state=RND_ST)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5, random_state=RND_ST)

Грузим стоп-слова, так как текст английский, то грузим английские

In [21]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Создаем счетчик, и считаем TF-IDF для корпуса текстов. fit запускаем только на обучающей иначе тестирование будет нечестным.

In [22]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train['lemm'])
features_valid = count_tf_idf.transform(features_valid['lemm'])
features_test = count_tf_idf.transform(features_test['lemm'])
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(79646, 100866)
(39823, 100866)
(39823, 100866)


In [23]:
model_LR = LogisticRegression()
train_f1 = cross_val_score(model_LR, features_train, target_train, cv=cv_counts, scoring='f1').mean()
print('F1 на кросс-валидации с несбалансированными классами', train_f1)

F1 на кросс-валидации с несбалансированными классами 0.6552225989020453


Теперь попробуем сбалансировать уменьшением весов классов

In [24]:
classes={0:1, 1:ratio}
model_LR = LogisticRegression(class_weight=classes)
train_f1_balanced = cross_val_score(model_LR, features_train, target_train, cv=cv_counts, scoring='f1').mean()
print('F1 на кросс-валидации с сбалансированными классами', train_f1_balanced)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


F1 на кросс-валидации с сбалансированными классами 0.7487778073326934


Посмотрим на AUC-ROC и решим помогла балансировка или нет

In [25]:
model_LR = LogisticRegression()
model_LR.fit(features_train, target_train)
probabilities_valid = model_LR.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc = roc_auc_score(target_valid, probabilities_one_valid)
valid_f1 = f1_score(target_valid, model_LR.predict(features_valid))


model_LR = LogisticRegression(class_weight=classes)
model_LR.fit(features_train, target_train)
probabilities_valid = model_LR.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_balanced = roc_auc_score(target_valid, probabilities_one_valid)
valid_f1_balanced = f1_score(target_valid, model_LR.predict(features_valid))

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

In [26]:
print('F1 на валидации с несбалансированными классами:', valid_f1)
print('F1 на валидации с сбалансированными классами:', valid_f1_balanced)
print('ROC-AUC на валидации с несбалансированными классами:', roc_auc)
print('ROC-AUC на валидации с сбалансированными классами:', roc_auc_balanced)

F1 на валидации с несбалансированными классами: 0.7228658536585366
F1 на валидации с сбалансированными классами: 0.74738714698688
ROC-AUC на валидации с несбалансированными классами: 0.9675866481852536
ROC-AUC на валидации с сбалансированными классами: 0.9696374098159376


Очень хорошие показатели ROC-AUC на сбалансированных и несбалансированных данных. Балансировка немного помогла. Далее будем применять сбалансированные.

## Обучение

С помощью GridSearchCV подберем параметры для LogisticRegression

In [27]:
%%time

classificator = LogisticRegression()
hyperparams = [{'solver':['newton-cg', 'liblinear'],'C':[0.1, 1, 10],'class_weight':[classes]}]

grd = GridSearchCV(classificator, hyperparams, scoring='f1',cv=cv_counts)
grd.fit(features_train, target_train)
print("Лучшие параметры:")
print()
LR_best_params = grd.best_params_
print(LR_best_params)

Лучшие параметры:

{'C': 10, 'class_weight': {0: 1, 1: 8.841344371679229}, 'solver': 'newton-cg'}
CPU times: user 34.4 s, sys: 1min 11s, total: 1min 45s
Wall time: 1min 45s


Обучим LogisticRegression

In [28]:
%%time

model_1 = LogisticRegression()
model_1.set_params(**LR_best_params)
model_1.fit(features_train, target_train)
predict_1 = model_1.predict(features_valid)
cv_f1_LR = cross_val_score(model_1,features_train, target_train, cv=cv_counts, scoring='f1').mean()
valid_f1_LR = f1_score(target_valid, predict_1)
print('F1 на кросс-валидации', cv_f1_LR)
print('F1 на валидации', valid_f1_LR)

F1 на кросс-валидации 0.7579091344825816
F1 на валидации 0.7534795942439254
CPU times: user 19 s, sys: 32.1 s, total: 51 s
Wall time: 51.1 s


Обучим CatBoostClassifier

In [29]:
%%time

model_2 = CatBoostClassifier(verbose=False, iterations=250)
model_2.fit(features_train, target_train)
predict_2 = model_2.predict(features_valid)
cv_f1_CBC = cross_val_score(model_2,features_train, target_train, cv=cv_counts, scoring='f1').mean()
valid_f1_CBC = f1_score(target_valid, predict_2)
print('F1 на кросс-валидации', cv_f1_CBC)
print('F1 на валидации', valid_f1_CBC)

F1 на кросс-валидации 0.7217702519055585
F1 на валидации 0.7434661994451743
CPU times: user 12min 24s, sys: 1min 43s, total: 14min 8s
Wall time: 14min 11s


CatBoostClassifier возможно сможет показать себя лучше при большем кол-ве итераций, но это долго

Выведем сводную таблицу по результатам

In [30]:
index = ['LogisticRegression','CatBoostClassifier']
data = {'F1 на кросс-валидации':[cv_f1_LR,cv_f1_CBC],'F1 на валидации':[valid_f1_LR,valid_f1_CBC]}

scores = pd.DataFrame(data=data, index=index)
scores['Выполнение задачи'] = scores['F1 на валидации'] > 0.75
scores

,F1 на кросс-валидации,F1 на валидации,Выполнение задачи
LogisticRegression,0.757909,0.753480,True
CatBoostClassifier,0.721770,0.743466,False


## Выводы

In [31]:
%%time

model_test = LogisticRegression()
model_test.set_params(**LR_best_params)
model_test.fit(features_train, target_train)
probabilities_test = model_test.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
predict_test = model_test.predict(features_test)
print('ROC-AUC:', roc_auc_score(target_test, probabilities_one_test))
print('F1:', f1_score(target_test, predict_test))

ROC-AUC: 0.9623142793336896
F1: 0.7598346131128176
CPU times: user 8.46 s, sys: 17.9 s, total: 26.4 s
Wall time: 26.4 s


LogisticRegression оказалась наилучшей моделью с f1_score на тестовой выборке = 0.76 и roc_auc на тестовой выборке = 0.96 , что соотвествует условию.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны